In [1]:
import tensorflow as tf
import tensorflow_text as tf_text
import numpy as np
import pandas as pd
import json
import FTokenizer

In [2]:
with open("E:/MLCourse/foxBot/foxPython/word_2_vec/data/new_vocabs_weights4.json","r",encoding="utf-8") as f:
    vocab_weights=json.load(f)
len(vocab_weights)

with open("E:/MLCourse/foxBot/foxPython/data/bi_grams.txt",encoding="utf-8") as f:
    bi_gramtxt=f.read()
bi_gram_list=[]
for each in bi_gramtxt.split(","):
    each=each[:-1]
    each=each[2:]
    each=each.replace(" ","_")
    bi_gram_list.append(each)

In [3]:
with open("E:/MLCourse/foxBot/foxPython/data/vi_vtb-ud-train-dev.json","r",encoding="utf-8") as f:
    data=json.load(f)
print(data[0]["paragraphs"][0]["sentences"][0]["tokens"])

with open("E:/MLCourse/foxBot/foxPython/data/vi_vtb-ud-test.json","r",encoding="utf-8") as f:
    test_data=json.load(f)
print(test_data[0]["paragraphs"][0]["sentences"][0]["tokens"])

[{'id': 0, 'orth': 'mảnh', 'tag': 'Nc', 'pos': 'NOUN', 'head': 1, 'dep': 'compound'}, {'id': 1, 'orth': 'đất', 'tag': 'N', 'pos': 'NOUN', 'head': 5, 'dep': 'nsubj'}, {'id': 2, 'orth': 'của', 'tag': 'E', 'pos': 'ADP', 'head': 1, 'dep': 'case'}, {'id': 3, 'orth': 'đạn', 'tag': 'N', 'pos': 'NOUN', 'head': -2, 'dep': 'nmod'}, {'id': 4, 'orth': 'bom', 'tag': 'N', 'pos': 'NOUN', 'head': -1, 'dep': 'compound'}, {'id': 5, 'orth': 'không', 'tag': 'R', 'pos': 'X', 'head': 1, 'dep': 'advmod'}, {'id': 6, 'orth': 'còn', 'tag': 'V', 'pos': 'VERB', 'head': 0, 'dep': 'ROOT'}, {'id': 7, 'orth': 'người', 'tag': 'N', 'pos': 'NOUN', 'head': -1, 'dep': 'obj'}, {'id': 8, 'orth': 'nghèo', 'tag': 'A', 'pos': 'ADJ', 'head': -1, 'dep': 'amod'}, {'id': 9, 'orth': '.', 'tag': '.', 'pos': 'PUNCT', 'head': -3, 'dep': 'punct'}]
[{'id': 0, 'orth': 'giờ', 'tag': 'N', 'pos': 'PUNCT', 'head': 3, 'dep': 'nsubj'}, {'id': 1, 'orth': 'G', 'tag': 'Ny', 'pos': 'NOUN', 'head': -1, 'dep': 'compound'}, {'id': 2, 'orth': 'đã', 't

In [4]:
# Convert từ list từ qua câu (TRAIN DATASET)
count=0
orth=[]
tag=[]
pos=[]
head=[]
dep=[]
for i in data:
    for j in i["paragraphs"][0]["sentences"][0]["tokens"]:
        orth.append(j["orth"])
        tag.append(j["tag"])
        pos.append(j["pos"])
        head.append(j["head"])
        dep.append(j["dep"])
df=pd.DataFrame({"orth":orth,"tag":tag,"pos":pos,"head":head,"dep":dep})
df.loc[df["pos"]=="NOUN"].head(30)

token=""
train_sentences=[]
train_postokenslist=[]
train_deptokenslist=[]
postoken=[]
deptoken=[]
datadict={}
myData=[]
for i,j,z in zip(orth,pos,dep):
    if j!="PUNCT":
        token=token+i+" "
        postoken.append(j)
        deptoken.append(z)
    if i==".":
        train_sentences.append(token)
        train_postokenslist.append(postoken)
        train_deptokenslist.append(deptoken)
        deptoken=[]
        postoken=[]
        token=""

In [6]:
# Convert từ list từ qua câu (TEST DATASET)
count=0
orth=[]
tag=[]
pos=[]
head=[]
dep=[]
for i in test_data:
    for j in i["paragraphs"][0]["sentences"][0]["tokens"]:
        orth.append(j["orth"])
        tag.append(j["tag"])
        pos.append(j["pos"])
        head.append(j["head"])
        dep.append(j["dep"])
test_df=pd.DataFrame({"orth":orth,"tag":tag,"pos":pos,"head":head,"dep":dep})
test_df.loc[df["pos"]=="NOUN"].head(30)

token=""
test_sentences=[]
test_postokenslist=[]
test_deptokenslist=[]
postoken=[]
deptoken=[]
datadict={}
myData=[]
for i,j,z in zip(orth,pos,dep):
    if j!="PUNCT":
        token=token+i+" "
        postoken.append(j)
        deptoken.append(z)
    if i==".":
        test_sentences.append(token)
        test_postokenslist.append(postoken)
        test_deptokenslist.append(deptoken)
        deptoken=[]
        postoken=[]
        token=""

In [7]:
#Convert hôm nay -> hôm_nay
t=[]
for i in df["orth"].tolist():
    for each in i:
        if each=="_":
            t.append(i.lower())
            break
t

['chiến_tranh',
 'quê_hương',
 'thái_mỹ',
 'củ_chi',
 'gia_đình',
 'chính_sách',
 'thái_mỹ',
 'mất_mát',
 'liệt_sĩ',
 'bom_đạn',
 'gia_đình',
 'buôn_lậu',
 'thái_mỹ',
 'bây_giờ',
 'trở_mình',
 'đồn_bót',
 'cùi_tay',
 'lạ_đời',
 'xe_máy',
 'băng_băng',
 'phẳng_lì',
 'tất_cả',
 'nội_đồng',
 'thành_quả',
 'nhà_nước',
 'nhân_dân',
 'tự_hào',
 'tình_nguyện',
 'thu_hẹp',
 'người_người',
 'ai_nấy',
 'cựu_chiến_binh',
 'nhẹ_tênh',
 'cuộc_sống',
 'thái_mỹ',
 'học_sinh',
 'xanh_ngắt',
 'thấp_thoáng',
 'dây_điện',
 'gia_đình',
 'không_những',
 'ban_đêm',
 'sáng_choang',
 'dự_án',
 'thủy_lợi',
 'trị_giá',
 'ra_đời',
 'vô_vàn',
 'nhọc_nhằn',
 'thi_đua',
 'mặt_đất',
 'đồn_bót',
 'trơ_trơ',
 'mồ_hôi',
 'bằng_không',
 'buôn_lậu',
 'đan_lát',
 'thoi_thóp',
 'bắt_đầu',
 'bảo_lãnh',
 'vận_động',
 'trở_về',
 'cải_tạo',
 'khai_hoang',
 'vận_động',
 'hồi_sinh',
 'đường_đất',
 'vận_động',
 'vận_động',
 'tháo_gỡ',
 'thay_da_đổi_thịt',
 'thái_mỹ',
 'nhân_dân',
 'đảng_viên',
 'thành_viên',
 'hội_đoàn',
 'mạch_m

In [8]:
for i in t:
    if bi_gram_list.count(i)==0:
        bi_gram_list.append(i)

In [11]:
rsen=[]
for i in train_sentences:
    rsen.append(i[:-1])

test_rsen=[]
for i in test_sentences:
    test_rsen.append(i[:-1])

In [54]:
posdata={}
posdata["messages"]=rsen
posdata["pos"]=train_postokenslist
# with open("E:/MLCourse/TF2/Notebook/tutorialNotebook/NewTagger/posdata.json","w",encoding="utf-8") as f:
#     json.dump(posdata,f)

In [66]:
VNTokenizer=tf.keras.preprocessing.text.Tokenizer(20000,filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n',oov_token="<UNK>",lower=True)
VNTokenizer.fit_on_texts(train_sentences)
seqTokenized=VNTokenizer.texts_to_sequences(train_sentences)
final_seq=tf.keras.preprocessing.sequence.pad_sequences(seqTokenized)
maxseqlen=final_seq.shape[1]
test_seqTokenized=VNTokenizer.texts_to_sequences(test_sentences)
final_test_seq=tf.keras.preprocessing.sequence.pad_sequences(test_seqTokenized,maxseqlen)


depTokenizer=tf.keras.preprocessing.text.Tokenizer(20000)
depTokenizer.fit_on_texts(train_deptokenslist)
depTokenized=depTokenizer.texts_to_sequences(train_deptokenslist)
final_dep=tf.keras.preprocessing.sequence.pad_sequences(depTokenized)

posTokenizer=tf.keras.preprocessing.text.Tokenizer(20000)
posTokenizer.fit_on_texts(train_postokenslist)
posTokenized=posTokenizer.texts_to_sequences(train_postokenslist)
test_pos_Tokenized=posTokenizer.texts_to_sequences(test_postokenslist)
final_pos=tf.keras.preprocessing.sequence.pad_sequences(posTokenized)
final_test_pos=tf.keras.preprocessing.sequence.pad_sequences(test_pos_Tokenized,maxlen=final_pos.shape[1])


In [67]:
final_seq.shape

(1969, 71)

In [14]:
class FTokenizer():
    def __init__(self,bi_gram_list):
        self.bi_gram_list=bi_gram_list

    def bi_gram_checker(self,bi_gram_word):
        for i in self.bi_gram_list:
            if (bi_gram_word==i):
                return True
        return False
    
    def fit_on_text(self,data):
        word_to_index={}
        index_to_word={}
        word_to_index["<PAD>"]=0
        word_to_index["<OOV>"]=1
        index_to_word[0]="<PAD>"
        index_to_word[1]="<OOV>"
        myContinue=False
        tokens=[]
        for eachseq in data:
            listtokens=eachseq.lower().split(" ")
            listtokens.append("")
            for i in range(0,len(listtokens)-1):
                if (myContinue):
                    myContinue=False
                    continue
                if (listtokens[i+1]!=""):
                    pairtext=listtokens[i]+"_"+listtokens[i+1]
                    if (self.bi_gram_checker(pairtext)):
                        myContinue=True
                        if (tokens.count(pairtext)==0):
                            tokens.append(pairtext)
                if (myContinue==False):  
                    if (tokens.count(listtokens[i])==0):
                        tokens.append(listtokens[i])
        for index,word in enumerate(tokens):
            word_to_index[word]=index+2
            index_to_word[index+2]=word
        self.word_to_index=word_to_index
        self.index_to_word=index_to_word

    def fit_on_text2(self,data):
        word_to_index={}
        index_to_word={}
        word_to_index["<PAD>"]=0
        word_to_index["<OOV>"]=1
        index_to_word[0]="<PAD>"
        index_to_word[1]="<OOV>"
        myContinue=False
        tokens=[]
        for eachseq in data:
            listtokens=eachseq.lower().split(" ")
            listtokens.append("")
            for i in range(0,len(listtokens)-1):
                if (listtokens[i+1]!=""):
                    pairtext=listtokens[i]+"_"+listtokens[i+1]
                    if (self.bi_gram_checker(pairtext)):
                        if (tokens.count(pairtext)==0):
                            tokens.append(pairtext)
                if (tokens.count(listtokens[i])==0 and listtokens[i]!=""):
                    tokens.append(listtokens[i])
        for index,word in enumerate(tokens):
            word_to_index[word]=index+2
            index_to_word[index+2]=word
        self.word_to_index=word_to_index
        self.index_to_word=index_to_word
    
    def get_word_to_index(self):
        return self.word_to_index
    
    def get_index_to_word(self):
        return self.index_to_word

    def get_max_length(self,data):
        maxsen=0
        for i in data:
            if maxsen<len(i):
                maxsen=len(i)
        return maxsen

    def text_to_sequence(self,textdata):
        d_continue=False
        output=[]
        for text in textdata:
            text=text.lower().split(" ")
            text.append("")
            sth=[]
            d_continue=False
            for i in range(0,len(text)-1):
                if d_continue==True:
                    d_continue=False
                    continue
                if (text[i+1]!=""):
                    pairtext=text[i]+"_"+text[i+1]
                    for j in self.word_to_index:
                        if pairtext.lower()==j:
                            sth.append(self.word_to_index[j])
                            d_continue=True
                            break
                if d_continue==False:
                    for count,j in enumerate(self.word_to_index):
                        if text[i].lower()==j:
                            sth.append(self.word_to_index[j])
                            break
                        if count==len(self.word_to_index)-1:
                            sth.append(1)
            output.append(sth)
        return output
    
    def intseq_to_index(self,sequence):
        output=[]
        for i in sequence:
            output.append(self.index_to_word[i])
        return output

    def pad_sequence(self,datasequence,maxlen):
        for i in datasequence:
            while len(i)!=maxlen:
                i.insert(0,0)
        return datasequence

    def bi_gram_ize(self,text):
        listtokens=text.lower().split(" ")
        listtokens.append("")
        myContinue=False
        tokens=[]
        for i in range(0,len(listtokens)-1):
            if (myContinue):
                myContinue=False
                continue
            if (listtokens[i+1]!=""):
                pairtext=listtokens[i]+"_"+listtokens[i+1]
                if (self.bi_gram_checker(pairtext)):
                    myContinue=True
                    if (tokens.count(pairtext)==0):
                        tokens.append(pairtext)
            if (myContinue==False):  
                if (tokens.count(listtokens[i])==0):
                    tokens.append(listtokens[i])
        return tokens


In [16]:
myTokenizer=FTokenizer(t)
myTokenizer.fit_on_text2(rsen)
seqTokenized2=myTokenizer.text_to_sequence(rsen)
test_seqTokenized2=myTokenizer.text_to_sequence(test_rsen)
final_seq2=tf.keras.preprocessing.sequence.pad_sequences(seqTokenized2)
mlen2=final_seq2.shape[1]
final_test_seq2=tf.keras.preprocessing.sequence.pad_sequences(test_seqTokenized2,maxlen=final_seq2.shape[1])

In [ ]:
# num_tokens = len(myTokenizer.word_to_index)
# embedding_dim = 128
# hits = 0
# misses = 0

# # Prepare embedding matrix
# embedding_matrix = np.zeros((num_tokens, embedding_dim))
# for word, i in myTokenizer.word_to_index.items():
#     embedding_vector = vocab_weights.get(word)
#     if embedding_vector is not None:
#         # Words not found in embedding index will be all-zeros.
#         # This includes the representation for "padding" and "OOV"
#         embedding_matrix[i] = embedding_vector
#         hits += 1
#     else:
#         misses += 1
# print("Converted %d words (%d misses)" % (hits, misses))

# embedding_layer = tf.keras.layers.Embedding(
#     num_tokens,
#     embedding_dim,
#     embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
#     trainable=False,
# )

Converted 2332 words (2031 misses)


In [84]:
def LSTMModel():
    inputs=tf.keras.layers.Input(shape=(maxseqlen,),dtype="int32")
    embedding=tf.keras.layers.Embedding(len(VNTokenizer.word_index)+1,128)(inputs)
    # embedding=embedding_layer(inputs)
    bilstmlayer=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True))(embedding)
    dense=tf.keras.layers.Dense(16,activation="relu")(bilstmlayer)
    output=tf.keras.layers.Dense(len(posTokenizer.word_index)+1,activation="softmax")(dense)
    return tf.keras.Model(inputs,output)

In [ ]:
final_test_pos.shape

(722, 71)

In [85]:
myModel=LSTMModel()
myModel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="sparse_categorical_crossentropy",metrics=["sparse_categorical_accuracy"])
myModel.fit(final_seq,final_pos,batch_size=32,epochs=100,validation_data=(final_test_seq,final_test_pos))

Epoch 1/100
62/62 [==============================] - 6s 36ms/step - loss: 0.8363 - sparse_categorical_accuracy: 0.8265 - val_loss: 0.4637 - val_sparse_categorical_accuracy: 0.8716
Epoch 2/100
62/62 [==============================] - 2s 26ms/step - loss: 0.4066 - sparse_categorical_accuracy: 0.8755 - val_loss: 0.4039 - val_sparse_categorical_accuracy: 0.8745
Epoch 3/100
62/62 [==============================] - 1s 23ms/step - loss: 0.3704 - sparse_categorical_accuracy: 0.8771 - val_loss: 0.3803 - val_sparse_categorical_accuracy: 0.8746
Epoch 4/100
62/62 [==============================] - 1s 22ms/step - loss: 0.3412 - sparse_categorical_accuracy: 0.8822 - val_loss: 0.3408 - val_sparse_categorical_accuracy: 0.8925
Epoch 5/100
62/62 [==============================] - 1s 23ms/step - loss: 0.2722 - sparse_categorical_accuracy: 0.9184 - val_loss: 0.2595 - val_sparse_categorical_accuracy: 0.9220
Epoch 6/100
62/62 [==============================] - 1s 23ms/step - loss: 0.1864 - sparse_categorica

In [63]:
final_seq.shape

(1969, 71)

In [75]:
VNTokenizer.index_word[0]="<PAD>"
def insert_text(inp):
    inp_gramized=myTokenizer.bi_gram_ize(inp)
    inp_tokenized=VNTokenizer.texts_to_sequences([inp_gramized])
    final_inp=tf.keras.preprocessing.sequence.pad_sequences(inp_tokenized,maxlen=maxseqlen)
    return final_inp
insert_text("Hôm nay quan trọng lắm đó").shape

(1, 71)

In [58]:
def insert_text(inp):
    inp_tokenized=myTokenizer.text_to_sequence(inp)
    final_inp=tf.keras.preprocessing.sequence.pad_sequences(inp_tokenized,maxlen=mlen2)
    return final_inp
insert_text(["ủa ai nấy"]).shape

(1, 69)

In [87]:
myModel.save("E:/MLCourse/foxBot/foxPython/NewTagger/PosTaggerModel")

INFO:tensorflow:Assets written to: E:/MLCourse/TF2/Notebook/tutorialNotebook/NewTagger/PosTaggerModel\assets


INFO:tensorflow:Assets written to: E:/MLCourse/TF2/Notebook/tutorialNotebook/NewTagger/PosTaggerModel\assets


In [55]:
posTokenizer.index_word

{1: 'noun',
 2: 'verb',
 3: 'x',
 4: 'adp',
 5: 'adj',
 6: 'propn',
 7: 'num',
 8: 'cconj',
 9: 'det',
 10: 'sconj',
 11: 'aux',
 12: 'part',
 13: 'intj',
 0: '<PAD>'}

In [86]:
posTokenizer.index_word[0]="<PAD>"
posPred=myModel.predict(insert_text("Phim bạn yêu thích là gì"))
np.round(posPred[0],decimals=2)
for i in posPred[0]:
    print(posTokenizer.index_word[np.argmax(i)],end=" ")

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> noun noun verb verb aux propn 